In [1]:
import numpy as np
from netCDF4 import Dataset as NF
import glob

# concatenate pyOM snapshots written by each processor

list = []
for file in glob.glob('snap_i=?????:?????_j=?????:?????.cdf'):
    is_ = int(file[file.find('i=')+2:file.find('i=')+7]) 
    ie_ = int(file[file.find('i=')+8:file.find('i=')+13]) 
    js_ = int(file[file.find('j=')+2:file.find('j=')+7]) 
    je_ = int(file[file.find('j=')+8:file.find('j=')+13])    
    a = {'name':file,'is':is_,'ie':ie_,'js':js_,'je':je_} 
    list.append(a)
    
nx =np.max( np.array(  [ a['ie'] for a in list]  ) )
ny =np.max( np.array(  [ a['je'] for a in list]  ) )

# read time and vertical axis
ifid = NF(list[0]['name'],mode='r')
t = ifid.variables['time'][:]
zt = ifid.variables['zt'][:]
zu = ifid.variables['zu'][:]
ifid.close()
nt,nz = t.shape[0],zt.shape[0]

print (nx,ny,nz,nt)

# read complete horizontal axis
xt = np.zeros( (nx,)  ,np.float32)
yt = np.zeros( (ny,)  ,np.float32)
xu = np.zeros( (nx,)  ,np.float32)
yu = np.zeros( (ny,)  ,np.float32)
print ('reading horizontal axis')
for a in list:
  ifid = NF(a['name'],mode='r')
  xt[ a['is']-1:a['ie']]  = ifid.variables['xt'][:]
  yt[ a['js']-1:a['je']]  = ifid.variables['yt'][:]
  xu[ a['is']-1:a['ie']]  = ifid.variables['xu'][:]
  yu[ a['js']-1:a['je']]  = ifid.variables['yu'][:]
  ifid.close()


print ("done")


768 512 64 38
reading horizontal axis
done


In [3]:
ifid = NF(list[0]['name'],mode='r')

#outfile = 'snap_n=%08d.cdf'%(n,)
outfile = 'pyOM.cdf'
print ('writing to file',outfile)
ofid = NF(outfile,mode='w')
    
# define dimensions   
ofid.createDimension('xt',nx)
ofid.createDimension('xu',nx)
ofid.createDimension('yt',ny)
ofid.createDimension('yu',ny)
ofid.createDimension('zt',nz)
ofid.createDimension('zu',nz)
ofid.createDimension('time',nt)

# define and store grid variables   
for var in ['xt','yt','zt','xu','yu','zu','time']:    
  id = ofid.createVariable(var,np.float32,(var,) )
  for attr in ifid.variables[var].ncattrs():  
    setattr(id,attr, getattr( ifid.variables[var] ,attr))
 
ofid.variables['time'][:] = t[:]
ofid.variables['xt'][:] = xt[:]
ofid.variables['xu'][:] = xu[:]
ofid.variables['yt'][:] = yt[:]
ofid.variables['yu'][:] = yu[:]
ofid.variables['zt'][:] = zt[:]
ofid.variables['zu'][:] = zu[:]

try:
   spval = ifid.variables['u']._FillValue
except:
   spval = None 

def def_var(var,dims):
  id = ofid.createVariable(var,np.float32,dims,fill_value = spval) 
  for attr in ifid.variables[var].ncattrs():  
    if attr != '_FillValue': setattr(id,attr, getattr( ifid.variables[var] ,attr))
 
# define 2D and 3D variables
def_var('surf_press',('time','yt','xt'))  
#def_var('forc_temp_surface',('time','yt','xt'))
#def_var('forc_salt_surface',('time','yt','xt'))
#def_var('taux',('time','yt','xu'))
#def_var('tauy',('time','yu','xt'))

def_var('u',('time','zt','yt','xu')) 
def_var('v',('time','zt','yu','xt'))  
def_var('w',('time','zu','yt','xt'))  
def_var('temp',('time','zt','yt','xt'))  
#def_var('salt',('time','zt','yt','xt'))  
def_var('press',('time','zt','yt','xt')) 


if 0: # low memory consumption, slow version
 print ('allocating memory')    
 # allocate memory                                                 
 var3D = np.zeros( (nz,ny,nx)  ,np.float32)
 var2D = np.zeros( (ny,nx)  ,np.float32)
 for n in range(nt):
  print ('n=',n,'/',nt)       
  # loop over files, read and write 2D and 3D variables
  for var in ['surf_press',]:
   print ('reading ',var)
   for a in list:
     var2D[a['js']-1:a['je'], a['is']-1:a['ie']]  = NF(a['name'],mode='r').variables[var][n,:]      
   print ("writing ",var)
   ofid.variables[var][n,:] = var2D[:]
  for var in ['u','v','w','temp','press']:
   print ('reading ',var)
   for a in list:
    var3D[ :, a['js']-1:a['je'], a['is']-1:a['ie']]  = NF(a['name'],mode='r').variables[var][n,:]      
   print ("writing ",var)
   ofid.variables[var][n,:] = var3D[:]

if 1: # high memory consumption, faster version
 print ('allocating memory')    
 # allocate memory                                                 
 var3D = np.zeros( (nt,nz,ny,nx)  ,np.float32)
 var2D = np.zeros( (nt,ny,nx)  ,np.float32)      
 # loop over files, read and write 2D and 3D variables
 for var in ['surf_press',]:
   print ('reading ',var)
   for a in list:
     var2D[:,a['js']-1:a['je'], a['is']-1:a['ie']]  = NF(a['name'],mode='r').variables[var][:nt,:]      
   print ("writing ",var)
   ofid.variables[var][:] = var2D[:]
 for var in ['u','v','w','temp','press']:
   print ('reading ',var)
   for a in list:
    var3D[:,:,a['js']-1:a['je'], a['is']-1:a['ie']]  = NF(a['name'],mode='r').variables[var][:nt,:]      
   print ("writing ",var)
   ofid.variables[var][:] = var3D[:]
         
ofid.close() 
ifid.close() 
print ("done")

writing to file pyOM.cdf
allocating memory
reading  surf_press


/tmp/ipykernel_2221239/3452393670.py:39: UserWarning: WARNING: missing_value cannot be safely cast to variable dtype
  if attr != '_FillValue': setattr(id,attr, getattr( ifid.variables[var] ,attr))


writing  surf_press
reading  u
writing  u
reading  v
writing  v
reading  w
writing  w
reading  temp
writing  temp
reading  press
writing  press
done
